In [1]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

## ～2021年10月19日

## 2021年10月19日～2021年10月27日

# 2021年11月16日～

## 下記のような処理を行う関数を作成する

関数名, モデル1, ... , モデルn

->

関数名, モデル1, ... , モデルn, MAPE最低値

## 下記のような処理を行う関数を作成する

{ベンチマーク名(str):MAPE表(DF)}

        
inputDict = ["benchmarkName":inputDF]

->

ベンチマーク名, MAPE平均

In [2]:
# 引数として渡されたDFに最低MAPEの列を追加する関数
# 引数：DF[関数名, モデル名1, ... , モデル名n]
# 返値：DF[関数名, モデル名1, ... , モデル名n, 最低MAPE]


def addLowestMAPEColumn(inputDF):

    funcNames = inputDF.index.to_list()
    modelNames = inputDF.columns.to_list()

    inputDF["最低値"] = math.inf

    for funcName in funcNames:
        lowestInFunc = math.inf
        seriesInFunc = inputDF.loc[funcName]
        for modelName in modelNames:
            elem = seriesInFunc[modelName]
            if elem < lowestInFunc:
                lowestInFunc = elem
            inputDF.at[funcName, "最低値"] = lowestInFunc

    return inputDF


# 引数として渡された辞書からDFを返す関数
# 引数：辞書{ベンチマーク名:DF[ベンチマーク名, 最低MAPEの平均]}
# 返値：DF[ベンチマーク名, 最低MAPEの平均]


def returnDFBenchmarkNameAndAverageLowestMAPE(inputDict):

    benchmarkNames = []
    lowestMAPEAverage = []

    for benchmarkName in inputDict.keys():
        averageSeries = inputDict[benchmarkName].mean()
        benchmarkNames.append(benchmarkName)
        lowestMAPEAverage.append(int(averageSeries["最低値"] * 10) / 10)

    resultDF = pd.DataFrame(
        index=benchmarkNames, data=lowestMAPEAverage, columns=["平均絶対誤差"]
    )
    return resultDF

In [3]:
# 発表用にテーブルをまとめたいので、DFを保持するリスト

resultsList = []

# コア数を128に固定し、問題サイズを振る

classes = ["A", "B", "C", "D", "E", "F"]
processes = [128]
dictColumnNamesJP = {
    "functionName": "関数名",
    "modelLin": "線形モデル",
    "modelIp": "反比例モデル",
    "modelLog": "対数モデル",
}
date = "2021年11月17日"

resultDict128WithoutInit = {}
resultDict128WithInit = {}

for benchmarkName in benchmarkNames:
    print(f"% _____benchmarkName={benchmarkName}_____")
    expVarBase = returnExplanatoryVariablesList(
        benchmarkName=benchmarkName,
        classes=classes,
        processes=processes,
        csvDirPath=csvDirPath,
        baseExpVar=True,
        initExpVar=False,
    )
    dfBase = returnDictAboutMAPETable(
        benchmarkName=benchmarkName,
        classes=classes,
        targetClass=None,
        processes=processes,
        targetProcess=None,
        expVar=expVarBase,
        csvDirPath=csvDirPath,
    )
    dfBase = dfBase.rename(columns=dictColumnNamesJP)
    dfBase = dfBase.set_index("関数名")
    # print(
    #     dfBase.to_latex(
    #         caption=f"初期変数を含めずに作成したモデルのMAPE（ベンチマーク{benchmarkName.upper()}（コア数128））",
    #         label=f"{date}BaseMAPE{benchmarkName}",
    #     )
    # )
    #
    resultDict128WithoutInit[benchmarkName.upper()] = addLowestMAPEColumn(dfBase)
    #
    expVarInit = returnExplanatoryVariablesList(
        benchmarkName=benchmarkName,
        classes=classes,
        processes=processes,
        csvDirPath=csvDirPath,
        baseExpVar=False,
        initExpVar=True,
    )
    dfInit = returnDictAboutMAPETable(
        benchmarkName=benchmarkName,
        classes=classes,
        targetClass=None,
        processes=processes,
        targetProcess=None,
        expVar=expVarInit,
        csvDirPath=csvDirPath,
    )
    dfInit = dfInit.rename(columns=dictColumnNamesJP)
    dfInit = dfInit.set_index("関数名")
    # print(
    #     dfInit.to_latex(
    #         caption=f"初期変数を含めて作成したモデルのMAPE（ベンチマーク{benchmarkName.upper()}（コア数128））",
    #         label=f"{date}InitMAPE{benchmarkName}",
    #     )
    # )
    #
    resultDict128WithInit[benchmarkName.upper()] = addLowestMAPEColumn(dfInit)
    #

csvDirPath = "./csv_files/"
test_returnExplanatoryVariablesList(csvDirPath=csvDirPath)

#
print(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDict128WithInit).to_latex(
        caption=f"初期変数を含めて構築したモデルの平均MAPE", label="128WithInit"
    )
)
resultsList.append(returnDFBenchmarkNameAndAverageLowestMAPE(resultDict128WithInit).copy)
print(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDict128WithoutInit).to_latex(
        caption=f"初期変数を含めずに構築したモデルの平均MAPE", label="128WithoutInit"
    )
)
resultsList.append(returnDFBenchmarkNameAndAverageLowestMAPE(resultDict128WithoutInit).copy)
#

# コア数と問題サイズの両方を振る

classes = ["A", "B", "C", "D", "E", "F"]
processes = [2, 4, 8, 16, 32, 64, 128, 256]
dictColumnNamesJP = {
    "functionName": "関数名",
    "modelLin": "線形モデル",
    "modelIp": "反比例モデル",
    "modelLog": "対数モデル",
}

resultDictAllWithInit = {}
resultDictAllWithoutInit = {}

for benchmarkName in benchmarkNames:
    print(f"% _____benchmarkName={benchmarkName}_____")
    expVarBase = returnExplanatoryVariablesList(
        benchmarkName=benchmarkName,
        classes=classes,
        processes=processes,
        csvDirPath=csvDirPath,
        baseExpVar=True,
        initExpVar=False,
    )
    dfBase = returnDictAboutMAPETable(
        benchmarkName=benchmarkName,
        classes=classes,
        targetClass=None,
        processes=processes,
        targetProcess=None,
        expVar=expVarBase,
        csvDirPath=csvDirPath,
    )
    dfBase = dfBase.rename(columns=dictColumnNamesJP)
    dfBase = dfBase.set_index("関数名")
    # print(
    #     dfBase.to_latex(
    #         caption=f"初期変数を含めずに作成したモデルのMAPE（ベンチマーク{benchmarkName.upper()}）",
    #         label=f"{date}BaseMAPE{benchmarkName}",
    #     )
    # )

    #
    resultDictAllWithoutInit[benchmarkName.upper()] = addLowestMAPEColumn(dfBase)
    #
    expVarInit = returnExplanatoryVariablesList(
        benchmarkName=benchmarkName,
        classes=classes,
        processes=processes,
        csvDirPath=csvDirPath,
        baseExpVar=False,
        initExpVar=True,
    )
    dfInit = returnDictAboutMAPETable(
        benchmarkName=benchmarkName,
        classes=classes,
        targetClass=None,
        processes=processes,
        targetProcess=None,
        expVar=expVarInit,
        csvDirPath=csvDirPath,
    )
    dfInit = dfInit.rename(columns=dictColumnNamesJP)
    dfInit = dfInit.set_index("関数名")
    # print(
    #     dfInit.to_latex(
    #         caption=f"初期変数を含めて作成したモデルのMAPE（ベンチマーク{benchmarkName.upper()}）",
    #         label=f"{date}InitMAPE{benchmarkName}",
    #     )
    # )
    #
    resultDictAllWithInit[benchmarkName.upper()] = addLowestMAPEColumn(dfInit)
    #

csvDirPath = "./csv_files/"
test_returnExplanatoryVariablesList(csvDirPath=csvDirPath)

#
print(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDictAllWithInit).to_latex(
        caption=f"初期変数を含めて構築したモデルの平均MAPE", label="AllWithInit"
    )
)
resultsList.append(returnDFBenchmarkNameAndAverageLowestMAPE(resultDictAllWithInit).copy)
print(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDictAllWithoutInit).to_latex(
        caption=f"初期変数を含めずに構築したモデルの平均MAPE", label="AllWithoutInit"
    )
)
resultsList.append(returnDFBenchmarkNameAndAverageLowestMAPE(resultDictAllWithoutInit).copy)
# .to_latex(caption=f"初期変数を含めて作成したモデルの平均MAPE")

% _____benchmarkName=cg_____
% _____benchmarkName=ep_____
% _____benchmarkName=ft_____
% _____benchmarkName=is_____
% _____benchmarkName=lu_____
% _____benchmarkName=mg_____
\begin{table}
\centering
\caption{初期変数を含めて構築したモデルの平均MAPE}
\label{128WithInit}
\begin{tabular}{lr}
\toprule
{} &  平均絶対誤差 \\
\midrule
CG &     0.0 \\
EP &     0.0 \\
FT &     0.0 \\
IS &     0.0 \\
LU &     1.6 \\
MG &     0.0 \\
\bottomrule
\end{tabular}
\end{table}

\begin{table}
\centering
\caption{初期変数を含めずに構築したモデルの平均MAPE}
\label{128WithoutInit}
\begin{tabular}{lr}
\toprule
{} &  平均絶対誤差 \\
\midrule
CG &    71.0 \\
EP &     0.0 \\
FT &    50.2 \\
IS &     0.0 \\
LU &    30.1 \\
MG &    10.4 \\
\bottomrule
\end{tabular}
\end{table}

% _____benchmarkName=cg_____
% _____benchmarkName=ep_____
% _____benchmarkName=ft_____
% _____benchmarkName=is_____
% _____benchmarkName=lu_____
% _____benchmarkName=mg_____
\begin{table}
\centering
\caption{初期変数を含めて構築したモデルの平均MAPE}
\label{AllWithInit}
\begin{tabular}{lr}
\toprule
{} &  平

In [4]:
class ModelLogForMultipleRegression(ModelBaseForMultipleRegression):
    # 対数モデル（重回帰分析）

    def inverterLog10Func(x):
        return 10 ** x

    def transformDataForModel(self, inputDF):
        # inputDFで与えられたデータをモデルに適した形に変形する
        returnDF = self.transformerLog10.transform(inputDF)
        return returnDF

    def setUpDataBeforeCalcLr(self):
        # 説明変数・目的変数を変換する関数
        self.transformerLog10 = sp.FunctionTransformer(
            inverse_func=self.inverterLog10Func, func=np.log10
        )
        # モデル構築用データ
        self.dataXForPredict = self.transformDataForModel(self.rawExplanaoryVariable)
        self.dataTForPredict = self.rawResponseVariable
        #         self.dataTForPredict = self.transformDataForModel(
        #             self.rawResponseVariable)
        # テスト用データ
        self.dataXForTest = self.transformDataForModel(
            self.rawExplanaoryVariableForTest
        )
        self.dataTForTest = self.rawResponseVariableForTest

    #         self.dataTForTest = self.transformDataForModel(
    #             self.rawResponseVariableForTest)

    def calcLr(self):
        # 実際にモデルを構築する
        self.lr = LinearRegression()
        self.lr.fit(self.dataXForPredict, self.dataTForPredict)

    def predict(self, inputDF):
        # inputDFのデータから構築されたモデルを使って予測を行う

        # inputDFから説明変数データのみを取得
        inputDFOnlyExplanatoryVariableColumn = inputDF[
            self.explanatoryVariableColumnNames
        ]
        # inputDFで与えられたデータをモデルに適した形に変形する
        transformedInputDF = self.transformDataForModel(
            inputDFOnlyExplanatoryVariableColumn
        )
        # 予測を実行
        result = self.lr.predict(transformedInputDF)

        return result

    pass


def test_ModelLogForMultipleRegressiont():
    
    # 説明変数
    plotX = np.linspace(0, 20, 10)
    plotY = np.linspace(20, 40, 10)
    plotZ = np.linspace(40, 60, 10)
    # 目的変数
    plotT = 1 * plotX + 2 / plotY + 3 * np.log10(plotZ) + 4

    # DFを作成する
    # カラム名のリスト
    columnNames = ["plotX", "plotY", "plotZ", "plotT"]
    datumForDF = [plotX, plotY, plotZ, plotT]
    inputDFForTest = pd.DataFrame(index=columnNames, data=datumForDF).T
    inputDFForTest["functionName"] = "functionName"

    # 目的変数・説明変数のカラム名のリスト
    # 目的変数のカラム名のリスト
    columnNamesForExp = columnNames[:-1]
    # 説明変数のカラム名のリスト
    columnNamesForRes = columnNames[-1:]

    # 予測をする
    # モデルオブジェクトの作成
    objectModel = ModelLogForMultipleRegression(
        inputDF=inputDFForTest,
        explanatoryVariableColumnNames=columnNamesForExp,
        responseVariableColumnNames=columnNamesForRes,
        conditionDictForTest={},
    )
    # モデルの生成の準備
    objectModel.setUpDataBeforeCalcLr()
    # モデルの生成
    objectModel.calcLr()
    # モデルによる予測
    # 入力データDFを作成
    inputDFForPredict = pd.DataFrame(inputDFForTest.tail(1))
    predictedNum = objectModel.predict(inputDFForPredict)

    # 相対誤差率でテスト対象のデータが想定通りに動作しているかを判断する
    # 相対誤差率を計算するために実データを取得する
    realNum = plotT[-1]
    relativeErrorRate = returnRelativeErrorRate(
        realNum=realNum, predictedNum=predictedNum
    )
    assert relativeErrorRate < 1